# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import argparse
import os
import joblib
from pprint import pprint

from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.run import Run
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

This is a binary classification task that will predict death by heart failure.

These are the twelve features in the dataset:
- age (int): self explanatory
- anaemia (bool): whether there has been a decrease of red blood cells or hemoglobin
- creatinine_phosphokinase (int): level of the CPK enzyme in the blood in mcg/L
- diabetes (bool): whether the patient has diabetes
- ejection_fraction (int): percentage of blood leaving the heart at each contraction
- high_blood_pressure (bool): whether the patient has hypertension
- platelets (int): platelets in the blood in kiloplatelets/mL
- serum_creatinine (float): level of serum creatinine in the blood in mg/dL
- serum_sodium (int): level of serum sodium in the blood in mEq/L
- sex (int): female or male (binary)

In [ ]:
# acquiring data
ds = TabularDatasetFactory.from_delimited_files("https://raw.githubusercontent.com/eparamasari/ML_Engineer_ND_Capstone/main/data/heart_failure_clinical_records_dataset.csv")

In [ ]:
ws = Workspace.from_config()

# creating an experiment object
experiment_name = 'HeartFailureClassificationExp'

experiment = Experiment(ws, experiment_name)

In [ ]:
# Checking an existing compute cluster or starting one
compute_name = "ComputeCluster"
try:
    compute_target = ComputeTarget(ws, compute_name)
    print(compute_name+ " already exist.")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", min_nodes=1, max_nodes=5)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

## AutoML Configuration

Why the automl settings and cofiguration used were chosen.

In [ ]:
# setting up automl

automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric": 'accuracy'
}

automl_config = AutoMLConfig(
    task='classification',
    compute_target=compute_target,
    training_data=ds,
    label_column_name='DEATH_EVENT',
    n_cross_validations=5,
    **automl_settings
)

In [ ]:
# Submitting the experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:

RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:

# Getting the best run and model
best_run, fitted_model = remote_run.get_output()

# Printing the best run
print(best_run)

# Getting all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Printing all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

In [ ]:
# Saving the best model
best_automl_model = best_run.register_model(model_path='outputs/model.pkl', model_name='heart_failure_automl',
                        tags = {'Training context': 'Automated ML'},
                        properties = {'Accuracy': best_run_metrics['accuracy']})

print(best_automl_model)

In [ ]:
# Listing registered models to verify that the model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Downloading the environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

# Downloading the scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                   environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

In [ ]:

import json

#Importing test data
test_df = data.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('Class')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests # for http post request

# Set the content type
headers = {'Content-type': 'application/json'}

response = requests.post(service.scoring_uri, test_sample, headers=headers)

In [ ]:
# Printing results from the inference
print(response.text)

In [ ]:
# Printing ground truth labels
print(label_df)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()